In [ ]:
import os
import cv2
import open3d as o3d
import rootutils
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

rootutils.setup_root(
    os.path.abspath(''), indicator=['.git', 'pyproject.toml'], pythonpath=True
)

from src.stereo_inference import StereoInferenceOnnx
from src.stereo_calibrate import (
    read_stereo_calibration,
    read_camera_intrinsics,
)
from src.utils import (
    StereoImages,
    visualize_disparity,
    postprocess_disparity,
    create_point_cloud,
)

In [ ]:
# Load stereo calibration parameters
stereo_map_left, stereo_map_right = read_stereo_calibration(
    '../camera_configs/stereo_calibration.xml'
)
# Read camera intrinsics
K, distance_between_cameras = read_camera_intrinsics(
    '../camera_configs/left_camera_intrinsics.xml'
)
# Load stereo model
stereo_model = StereoInferenceOnnx('../models/fs_800_640.onnx')

In [ ]:
processor = StereoImages(stereo_map_left, stereo_map_right)
proc_result = processor.process(
    '../data/samples/left.png',
    '../data/samples/right.png',
    stereo_model.input_width,
    stereo_model.input_height
    )

left_tensor, right_tensor = proc_result['tensors']
scale = proc_result['scale']
mask_rectified = None

# show rectified images
plt.subplot(1, 2, 1)
plt.imshow(proc_result['processed_images'][0])
plt.title('Left Rectified Image')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(proc_result['processed_images'][1])
plt.title('Right Rectified Image')
plt.axis('off')
plt.show()

In [ ]:
# Stereo inference
disparity_map = stereo_model(left_tensor, right_tensor)

In [ ]:
# squeeze form 4D to 2D
disparity_map = disparity_map.squeeze()
# Visualize disparity map
disparity_map_viz = visualize_disparity(disparity_map, mask=mask_rectified)
# Postprocess disparity map
disparity_map_proc = postprocess_disparity(disparity_map, mask=mask_rectified)

# save disparity map
output_path = Path('../output/disp/')
output_path.mkdir(parents=True, exist_ok=True)
cv2.imwrite(
    str(output_path / 'disparity_map.png'),
    cv2.cvtColor(disparity_map_viz, cv2.COLOR_RGB2BGR),
)

# show disparity map with matplotlib
plt.imshow(disparity_map_viz)
plt.axis('off')
plt.show()

In [ ]:
# Create point cloud
pcd = create_point_cloud(
    disparity_map_proc, proc_result['processed_images'][0], K, distance_between_cameras, scale=scale
)

# Save point cloud to PLY file
output_path = Path('../output/pcd/')
output_path.mkdir(parents=True, exist_ok=True)
o3d.io.write_point_cloud(str(output_path / 'pointcloud.ply'), pcd)

In [ ]:
# Visualize point cloud using Open3D
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)
vis.get_render_option().point_size = 1.0
vis.get_render_option().background_color = np.array([0.5, 0.5, 0.5])
vis.run()
vis.destroy_window()